In [7]:
import time
import uuid
import boto3
import sagemaker

sess = sagemaker.Session()
region = boto3.Session().region_name
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

In [8]:
model_training_s3_uri = 's3://dsoaws/nyc-taxi-orig-cleaned-dropped-parquet-all-years-multiple-files-1TB'

### Configure the algorithm and training job

The Training Job hyperparameters are set. For more information on XGBoost Hyperparameters, 
see https://xgboost.readthedocs.io/en/latest/parameter.html.

In [9]:
container = sagemaker.image_uris.retrieve("xgboost", region, "1.5-1")

output_content_type = "parquet"

train_content_type = (
    "application/x-parquet" if output_content_type.upper() == "PARQUET"
    else "text/csv"
)
train_input = sagemaker.inputs.TrainingInput(
    s3_data=model_training_s3_uri,
    content_type=train_content_type,
    distribution='ShardedByS3Key',
    input_mode='FastFile'
)

hyperparameters = {
    "eta": "0.2",
    "gamma": "4",
    "max_depth": "5",
    "min_child_weight": "6",
    "num_round": "50",
    "objective": "reg:squarederror",
    "subsample": "0.7"
}

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


### Start the Training Job

The TrainingJob configurations are set using the SageMaker Python SDK Estimator, and which is fit using 
the training data from the Processing Job that was run earlier.

In [10]:
estimator = sagemaker.estimator.Estimator(
    container,
    role,
    hyperparameters=hyperparameters,
    instance_count=12,
    instance_type="ml.m5.24xlarge",
    volume_size=200,
    input_mode='FastFile'    
)

In [11]:
import time
import sagemaker

# m5.24xlarge (6 incl leader node) - 12.85 minutes
training_job_results = estimator.fit({"train": train_input})
print(training_job_results)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-03-09-06-48-44-732


ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: No S3 objects found under S3 URL "s3://dsoaws/nyc-taxi-orig-cleaned-dropped-parquet-all-years-multiple-files-10TB/**/**/**" given in input data source. Please ensure that the bucket exists in the selected region (us-east-1), that objects exist under that S3 prefix, and that the role "arn:aws:iam::079002598131:role/service-role/AmazonSageMaker-ExecutionRole-20220804T150518" has "s3:ListBucket" permissions on bucket "dsoaws".